In [3]:

SELECT_SUBJECTS=['abstract_narrative_understanding', 'anachronisms', 'analogical_similarity', 'analytic_entailment', 'authorship_verification', 'causal_judgment', 'cause_and_effect', 'checkmate_in_one', 'hindu_knowledge', 'hinglish_toxicity', 'hyperbaton', 'indic_cause_and_effect', 'intent_recognition', 'international_phonetic_alphabet_nli', 'intersect_geometry', 'irony_identification', 'kanji_ascii', 'kannada', 'key_value_maps', 'salient_translation_error_detection', 'snarks', 'social_iqa', 'social_support', 'sports_understanding', 'strange_stories', 'strategyqa', 'swahili_english_proverbs', 'swedish_to_german_proverbs', 'symbol_interpretation', 'temporal_sequences', 'logical_deduction', 'dark_humor_detection']

# SAMPLE_SUBJECTS = [item for item in _SUBJECTS if item not in set(SELECT_SUBJECTS)]


In [4]:
from datasets import load_dataset
mmlu_datas = {}

In [5]:
for Name in SELECT_SUBJECTS:
    mmlu_datas[Name] = load_dataset("tasksource/bigbench",Name,split='train[0:50]')
# for Name in SAMPLE_SUBJECTS:
#     mmlu_datas[Name] = load_dataset("tasksource/bigbench",Name,split='train[0:150]')

Found cached dataset bigbench (/Users/xiayu/.cache/huggingface/datasets/tasksource___bigbench/abstract_narrative_understanding/1.0.0/c5da5ac497141c7435da10444495b8577405d4ed01e524265b144a7063718c0c)
Found cached dataset bigbench (/Users/xiayu/.cache/huggingface/datasets/tasksource___bigbench/anachronisms/1.0.0/c5da5ac497141c7435da10444495b8577405d4ed01e524265b144a7063718c0c)


In [ ]:
def get_prompt_answer(data):
    total_len = len(data['multiple_choice_targets'])
    choice_txt = ""
    for i in range(total_len):
        choice_txt += chr(ord('A')+i)+". "+data['multiple_choice_targets'][i]+"\n"
    prompt = f'Provide the corresponding options or responses for the following questions.\n'
    inputs = data['inputs'].split('\n')
    prompt += f"Question: {data['inputs']}\n{choice_txt}Answer is:"
    answer_txt = chr(ord('A')+data['multiple_choice_scores'].index(1))
    return prompt,answer_txt
    

In [ ]:
import random
print("stage1")
mmlu_train = []
select_names = []
# prefix = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
for Name in mmlu_datas:
    if Name in ['auto_categorization']: continue
    print(Name)
    data_train = mmlu_datas[Name]
    if len(data_train[0]['multiple_choice_scores'])==0 : continue
    select_names.append(Name)
    print(data_train[0])
    random.seed(len(Name))
    random_mmlu = random.sample(range(len(data_train)),min(len(data_train),50))
    select_mmlu = []
    for num in random_mmlu:
        nd = {}
        tt = data_train[num]
        prompt,answer_txt = get_prompt_answer(tt)
        nd["instruction"] = f"{prompt}"
        nd["input"] = ""
        nd["output"] = answer_txt
        select_mmlu.append(nd)
    mmlu_train.extend(select_mmlu)

print(len(mmlu_train))

print(mmlu_train[0:2])
print(len(mmlu_train))

stage1
abstract_narrative_understanding
{'inputs': 'In what follows, we provide short narratives, each of which illustrates a common proverb. \nNarrative: The children had been sitting outside of the kitchen for nearly an hour, revelling in the wonderful smell of new cupcakes coming through the door. Eventually, two of the children decided that they could not be bothered to be there any more and got up to leave, despite the pleas of the two remaining children. Five minutes later their grandmother came out of the kitchen with a batch of cupcakes for them to test. As two of the children had gone, the two left got a double helping!\nThis narrative is a good illustration of the following proverb:', 'targets': ['Good things come to those that wait'], 'multiple_choice_targets': ['Once bitten, twice shy', "What's sauce for the goose is sauce for the gander", "Don't let the grass grow under your feet", 'Silence is golden', 'Good things come to those that wait'], 'multiple_choice_scores': [0, 0

In [ ]:
import json
with open(f'datas/bigbench_{len(mmlu_train)}_raw.json', 'w') as output_file:
    json.dump(mmlu_train, output_file, indent=4)
print(select_names)

NameError: name 'mmlu_train' is not defined

In [ ]:
len(mmlu_train)

1600